# Qwen3 0.6B 모델과 감성분석 모델을 합치기 위한 코드

### 1. 모델 가져오기

- huggingface의 transformers를 사용하여 가져올 것
- 1차로 제대로 가져와서 모델이 작동하는지 확인 후 LLM 폴더에 저장

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")
messages = [
    {"role": "user", "content": "오늘 사용자의 일기에서 관측되는 감정은 합계를 1이라고 했을 때 우울 0.7, 상처 0.17, 불안 0.1, 기쁨0.03 입니다. 이 사용자에게 심리 상담을 해주고 싶은데 어떤 방식으로 할 지 추천해주세요"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

tokenizer_config.json: 0.00B [00:00, ?B/s]

c:\Users\SSAFY\Desktop\WANG\Kcvenv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SSAFY\.cache\huggingface\hub\models--Qwen--Qwen3-0.6B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

<think>
Okay, the user asked, "Who are you?" I need to respond in a friendly and helpful way. Let me start by acknowledging their question. I should mention that I'm an AI


In [2]:
import os

model_path = "./qwen3 0.6B"
os.makedirs(model_path, exist_ok=True)

tokenizer.save_pretrained(model_path)
model.save_pretrained(model_path)

## 모델을 로컬로 가져온 후 사용 시

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = './Qwen'
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to("cpu")

print(model.device)

cpu


### 사용자 질문 입력

In [13]:
import time

messages = [
    {"role": "user", "content": "오늘 사용자의 일기에서 관측되는 감정은 합계를 1이라고 했을 때 우울 0.7, 상처 0.17, 불안 0.1, 기쁨0.03 입니다. 이 사용자에게 심리 상담을 해주고 싶은데 어떤 방식으로 할 지 추천해주세요. 답변은 한국어로 해주세요"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

start = time.time()
outputs = model.generate(**inputs, max_new_tokens=400)
end = time.time()
output_time = end - start

print('답변 소요 시간 : ', output_time)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

답변 소요 시간 :  52.96225595474243
<think>
Okay, let's see. The user provided some emotion percentages: 1 for happiness, 0.7 for sadness, 0.17 for anger, 0.1 for fear, and 0.03 for joy. Wait, the total should add up to 1. Let me check: 0.7 + 0.17 + 0.1 + 0.03 + 0.0? No, that adds up to 0.7 + 0.17 = 0.87, plus 0.1 is 0.97, plus 0.03 is 1.00. Oh right, the joy is 0.03. So the total is 1.00. 

Now, the user wants to give advice for a psychological consultation. First, I need to understand the emotions. Sadness is 0.7, which is quite high. Anger 0.17, fear 0.1, and joy 0.03. The user might be feeling overwhelmed or stressed. 

I should start by acknowledging the emotions. Then, maybe break them down. For sadness, suggest ways to process it, like journaling or talking. Anger could relate to stress, so perhaps recommend stress management techniques. Fear might be related to anxiety, so addressing that could help. 

Also, the percentages indicate that the user is experiencing a mix of emotions. It